In [15]:
import re
import os
import string
import pandas as pd
import pandas_profiling

pd.options.display.max_rows = 100
pd.options.display.max_columns = 70

# Tipos de campos a desconsiderar
typeNotConsider = ['date','datetime','string','text']
# Tabela para processar
table = "maint_order"
# maint_req , maint_order

#============================================

dir_csv = "./base_manusis/original/"
dir_save = "./base_manusis/preprocess/"

data = pd.read_csv(dir_csv+table+".csv",low_memory=False)

data['opened_at'] = pd.to_datetime(data['opened_at'], errors = 'coerce')
start_date = '01-01-2017'
end_date = '12-31-2019'
mask = (data['opened_at'] > start_date) & (data['opened_at'] <= end_date)
data = data.loc[mask]

# profile = pandas_profiling.ProfileReport(data)
# profile.to_file(output_file=dir_csv+table+"_describe.html")

f = open(dir_csv+table+".txt", "r",encoding='utf-8')
tt = f.read()
tt = tt.replace("\t", " ")
tt = tt.replace("\n", " ")
tt = tt.split('{')

k = []
for i in tt:
    y = []
    for t in i.split(','):
        u = t.strip()
        if any(c in u for c in ['desc','name','type']):
            try:
                u1 = u.split(':')
                u2 = u1[1].strip()
                if len(re.findall(r"(?<=\')(.*?)(?=\')",u2)) > 0:
                    u3 = re.findall(r"(?<=\')(.*?)(?=\')",u2)
                if len(re.findall(r'(?<=\")(.*?)(?=\")',u2)) > 0:
                    u3 = re.findall(r'(?<=\")(.*?)(?=\")',u2)
                else:
                    pass
                y.append(u3[0])
            except:
                pass
    if len(y) == 3:
        k.append(y)
    else:
        pass
    
data_extrac = pd.DataFrame(k)
data_extrac = data_extrac.rename(columns={data_extrac.columns[0]: 'desc',data_extrac.columns[1]: 'name',data_extrac.columns[2]: 'type'})
data_describe = data_extrac[['desc','type']].groupby(['type']).count()
columns = data_extrac['name'].tolist()

data_extrac = data_extrac[data_extrac['name'].isin(list(data.columns.values))]
select_columns = data_extrac[~data_extrac['type'].isin(typeNotConsider)]
select_columns = select_columns['name'].tolist()

data_select = data[select_columns].fillna(0)

list_columns = set(data_select.columns)
list_columns_select = []

for x in list_columns:
    data_select[x] = data_select[x].astype('category')
    if len(data_select[x].dtype.categories) > 1 and any(data_select[x].duplicated()):
        list_columns_select.append(x)
data_select = data_select[list_columns_select]

data_select = data_select.drop('estimated_worktime', axis=1)
data_select = data_select.drop('performed_asset_downtime', axis=1)
data_select = data_select.drop('performed_worktime', axis=1)
data_select = data_select.drop('priority_calculated', axis=1)
data_select = data_select.drop('maint_req_priority', axis=1)
data_select = data_select.drop('maint_req_id', axis=1)

list_exclude = ['cancel_user_id','origin_doc','prog_year_week_number','maint_order_cancel_reason_id',
                'total_cost','rpn','what','when','who','why','how','maint_order_on_hold_reason_id','performed_worktime',
                'estimated_worktime','performed_asset_downtime','id_mobile','mobile_uuid','rework_cos','index']

for i in list_exclude:
    try:
        data_select = data_select.drop(i, axis=1)
    except:
        pass
    
data_select.to_csv(dir_save+table+"_etl.csv", index=False)
profile = pandas_profiling.ProfileReport(data_select)
profile.to_file(outputfile=dir_save+table+"_describe.html")

In [3]:
dir_csv = "./base_manusis/original/"
dir_save = "./base_manusis/preprocess/"
import re
import os
import string
import pandas as pd
import pandas_profiling

table = "maint_order"
data = pd.read_csv(dir_csv+table+".csv",low_memory=False)

In [6]:
data.isna().sum()

id                                           0
order_number                                 0
company_id                                   0
area_id                                      0
first_loc_id                              6330
second_loc_id                             6349
asset_id                                  5370
asset_part_id                           378242
asset_part_child_id                     396850
asset_group_id                            5370
cost_center_id                            5147
maint_req_id                            332955
maint_service_type_id                        0
maint_service_nature_id                 312618
employee_team_id                        243061
employee_id                             216564
description                                  0
executed_service                        400512
maint_failure_id                        399953
maint_cause_id                          400125
maint_action_id                         400194
maint_order_s